In [ ]:
# 📦 Environment Setup
!pip install -q transformers datasets mlflow seqeval
!pip install -q git+https://github.com/huggingface/transformers.git  # Optional: latest
!git clone https://github.com/Teshager21/amharic-telegram-ecommerce-ner.git
%cd amharic-telegram-ecommerce-ner

# 📁 Mount Google Drive (Optional)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# ⚙️ DVC Setup (if needed to pull model/data from Google Drive)
!pip install -q dvc[dvc-gdrive]

In [ ]:
# 📎 Upload your service account JSON
from google.colab import files

uploaded = files.upload()  # Select your `dvc-drive-remote-XXXX.json`

In [ ]:
# 💾 Save and point DVC to service account key
import os
import json

filename = next(iter(uploaded))
key_path = f"/content/{filename}"

# Reconfigure the remote to use this JSON
!dvc remote modify gdrive_remote gdrive_service_account_json_file_path {key_path}

In [ ]:
# 🔄  Pull data/model using DVC
!dvc pull

In [ ]:
# Colab cell to run your fine-tuning main.py with Hydra config overrides

!python src/train/main.py \
  model.name_or_path=xlm-roberta-base \
  training.epochs=3 \
  training.batch_size=16 \
  data.conll_file=data/labeled/train.conll \
  output_dir=models/ner \
  logging.level=INFO


In [ ]:
# 🧠 Evaluate NER Model
from src.eval.evaluate import evaluate

LABEL_LIST = [
    "O",
    "B-PRODUCT", "I-PRODUCT",
    "B-PRICE", "I-PRICE",
    "B-LOC", "I-LOC",
]

evaluate(
    model_path="models/ner",  # or path to model in Google Drive
    data_path="data/processed/ner_test.conll",  # adjust as needed
    label_list=LABEL_LIST
)

In [ ]:
# 📊 View MLflow Run (Optional)
!ls -lh mlruns/